In [1]:
#請用python2打開
# 檔案路徑 可能要改一下
from sklearn.ensemble import RandomForestRegressor

# READ DATA
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt  

data_train= pd.read_csv("C:/COURSE_PYTHON_PANDAS_HOMEWORK/Titanic/train.csv")  #891rows
gender= pd.read_csv("C:/COURSE_PYTHON_PANDAS_HOMEWORK/Titanic/gender_submission.csv") #418rows
data_test= pd.read_csv("C:/COURSE_PYTHON_PANDAS_HOMEWORK/Titanic/test.csv")  #418rows
#waht is train data?
data_train[:1]
# #PassengerId：只是个乘客序号；
# Survived：最终是否存活；
# Pclass：舱位，1是头等舱，3是最低等，从电影里看，这个影响还是挺大的；
# Name：乘客姓名，除非是要算命，不然应该没啥影响；
# Sex：性别，应该影响很大；
# Age：年龄，有一部分数据缺失；
# SibSp：一同上船的兄弟姐妹或配偶；
# Parch：一同上船的父母或子女，目测这两项应该没啥影响吧，除非是要是一起死的那种；
# Ticket：船票信息，比较乱，完全看不出有任何用处；
# Fare：乘客票价，这个数据应该和Pclass有一定对应关系；
# Cabin：客舱编号
# Embarked：上船地点，主要是S（南安普顿）、C（瑟堡）、Q（皇后镇）


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S


In [2]:
import pandas as pd  

data=pd.read_csv('C:/COURSE_PYTHON_PANDAS_HOMEWORK/Titanic/train.csv')  

#剔除變量  

data.drop(['PassengerId','Ticket'],axis=1,inplace=True)  

#補全Embarked變量  

data.loc[data.Embarked.isnull(),'Embarked']='S'  

#one-hot編碼  

from sklearn.preprocessing import OneHotEncoder  

from sklearn.preprocessing import LabelEncoder  

#ohe_pclass=OneHotEncoder(sparse=False).fit(data[['Pclass']])  

#Pclass_ohe=ohe_pclass.transform(data[['Pclass']])  

le_sex=LabelEncoder().fit(data['Sex'])  

Sex_label=le_sex.transform(data['Sex'])  

ohe_sex=OneHotEncoder(sparse=False).fit(Sex_label.reshape(-1,1))  

Sex_ohe=ohe_sex.transform(Sex_label.reshape(-1,1))  

le_embarked=LabelEncoder().fit(data['Embarked'])  

Embarked_label=le_embarked.transform(data['Embarked'])  

ohe_embarked=OneHotEncoder(sparse=False).fit(Embarked_label.reshape(-1,1))  

Embarked_ohe=ohe_embarked.transform(Embarked_label.reshape(-1,1))  

def replace_name(x):  

    if 'Mrs' in x:  
        return 'Mrs'  

    elif 'Mr' in x:  

        return 'Mr'  

    else:  

        return 'Miss'  

data['Name']=data['Name'].map(lambda x:replace_name(x))  

le_name=LabelEncoder().fit(data['Name'])  

Name_label=le_name.transform(data['Name'])  

ohe_name=OneHotEncoder(sparse=False).fit(Name_label.reshape(-1,1))  

Name_ohe=ohe_name.transform(Name_label.reshape(-1,1))  

data['Sex_0']=Sex_ohe[:,0]  

data['Sex_1']=Sex_ohe[:,1]  

data['Embarked_0']=Embarked_ohe[:,0]  

data['Embarked_1']=Embarked_ohe[:,1]  

data['Embarked_2']=Embarked_ohe[:,2]  

data['Name_0']=Name_ohe[:,0]  

data['Name_1']=Name_ohe[:,1]  

data['Name_2']=Name_ohe[:,2]  

#歸一化處理  

from sklearn.preprocessing import StandardScaler  

Pclass_scale=StandardScaler().fit(data['Pclass'])  

data['Pclass_scaled']=StandardScaler().fit_transform(data['Pclass'].reshape(-1,1),Pclass_scale)  

Fare_scale=StandardScaler().fit(data['Fare'])  

data['Fare_scaled']=StandardScaler().fit_transform(data['Fare'].reshape(-1,1),Fare_scale)  

SibSp_scale=StandardScaler().fit(data['SibSp'])  

data['SibSp_scaled']=StandardScaler().fit_transform(data['SibSp'].reshape(-1,1),SibSp_scale)  

Parch_scale=StandardScaler().fit(data['Parch'])  

data['Parch_scaled']=StandardScaler().fit_transform(data['Parch'].reshape(-1,1),Parch_scale)  

#預測年紀並補全  

from sklearn.ensemble import RandomForestRegressor  

def set_missing_age(data):  

    train=data[['Age','SibSp_scaled','Parch_scaled','Name_0','Name_1','Name_2','Sex_0','Sex_1']]  

    known_age=train[train.Age.notnull()].as_matrix()  

    unknown_age=train[train.Age.isnull()].as_matrix()  

    y=known_age[:,0]  

    x=known_age[:,1:]  

    rf=RandomForestRegressor(random_state=0,n_estimators=200,n_jobs=-1)  

    rf.fit(x,y)  

    rf.score(x,y)  

    predictage=rf.predict(unknown_age[:,1:])  

    data.loc[data.Age.isnull(),'Age']=predictage  

    return data,rf  

data,rf=set_missing_age(data)  

Age_scale=StandardScaler().fit(data['Age'])  

data['Age_scaled']=StandardScaler().fit_transform(data['Age'].reshape(-1,1),Age_scale)  

train_x=data[['Sex_0','Sex_1','Embarked_0','Embarked_1','Embarked_2','Name_0','Name_1','Name_2','Pclass_scaled','Age_scaled','Fare_scaled']].as_matrix()  

train_y=data['Survived'].as_matrix()  


#模型構造  

from sklearn.model_selection import train_test_split  

from sklearn.linear_model import LogisticRegression  

x_tr,x_te,y_tr,y_te=train_test_split(train_x,train_y,test_size=0.3,random_state=0)  

lr=LogisticRegression(C=1.0,tol=1e-6)  

lr.fit(x_tr,y_tr)  

lr.score(x_te,y_te)  

from sklearn.svm import SVC  

svc=SVC(C=2, kernel='rbf', decision_function_shape='ovo')  

svc.fit(x_tr,y_tr)  

svc.score(x_te,y_te)  

from sklearn.ensemble import RandomForestClassifier  

randomf=RandomForestClassifier(n_estimators=500,max_depth=5,random_state=0)  

randomf.fit(x_tr,y_tr)  

randomf.score(x_te,y_te)  

from sklearn.ensemble import GradientBoostingClassifier  

gdbt=GradientBoostingClassifier(n_estimators=600,max_depth=5,random_state=0)  

gdbt.fit(x_tr,y_tr)  

gdbt.score(x_te,y_te)  

C:\ProgramData\Anaconda2\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)
C:\ProgramData\Anaconda2\lib\site-packages\sklearn\preprocessing\data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:84: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
C:\ProgramData\Anaconda2\lib\site-packages\sklearn\preprocessing\data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1

0.81343283582089554

In [3]:
pwd

u'C:\\Users\\Elisa'